In [ ]:
%%capture
!pip install highdicom
!pip install wsidicom
!pip install idc-index --upgrade
!pip install openslide-python

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
import json
import pandas as pd
import re
import os
import random
import subprocess
import pandas as pd
import numpy as np
import highdicom as hd
import matplotlib.pyplot as plt
from idc_index import index
from wsidicom import WsiDicom
from typing import List, Tuple, Union
import shapely
from shapely.geometry import Polygon, box
from shapely.strtree import STRtree
from PIL import Image, ImageDraw
from shapely.affinity import translate
from IPython.display import IFrame

In [ ]:
print(os.getcwd())

/content/gdrive/MyDrive/Capstone Code


In [ ]:
drive.mount('/content/gdrive')

path = "/content/gdrive/My Drive/Capstone Code/"
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
clinical = pd.read_json('./data/clinica_tcga_brca.json')

In [ ]:
manifest = pd.read_csv('./data/gdc_manifest.txt', sep='\t')

# Step 1: Split each gene symbol into three different dataframe from the clinical dataset



In [ ]:
## Function Extract the molecular tests from the clinical data
def extract_molecular_tests(follow_ups):
    """Extract all molecular tests from follow_ups data."""
    molecular_tests = []

    # Handle case where follow_ups might be None or NaN
    if follow_ups is None or (isinstance(follow_ups, float) and np.isnan(follow_ups)):
        return molecular_tests

    # Handle case where follow_ups is a list
    if isinstance(follow_ups, list):
        for follow_up in follow_ups:
            # Handle dictionary case
            if isinstance(follow_up, dict):
                if 'molecular_tests' in follow_up:
                    molecular_tests.extend(follow_up['molecular_tests'])
            # Handle list of dictionaries case
            elif isinstance(follow_up, list):
                for entry in follow_up:
                    if isinstance(entry, dict) and 'molecular_tests' in entry:
                        molecular_tests.extend(entry['molecular_tests'])

    # Handle case where follow_ups is a dictionary
    elif isinstance(follow_ups, dict):
        if 'molecular_tests' in follow_ups:
            molecular_tests.extend(follow_ups['molecular_tests'])

    return molecular_tests

In [ ]:
clinical['molecular_tests'] = clinical['follow_ups'].apply(extract_molecular_tests)

## 1.1 get all the positive and negative

In [ ]:
# change the extract_gene_data fucntion a little bit, change extract the test value range into only get the positive/negative test result of each gene symbol, one more thing is that there should be more than one test result, can you check all the test result but choose negative/positive from the table?

def extract_gene_data(clinical_data):
    """
    Extracts gene symbols and test results (positive/negative) from the 'molecular_tests' column.
    Considers all test results for each gene and prioritizes positive over negative.
    Returns NaN if no positive/negative results are found for a gene.
    """
    extracted_data = []
    for index, row in clinical_data.iterrows():
        submitter_id = row['submitter_id']
        molecular_tests = row['molecular_tests']
        gene_data = {'submitter_id': submitter_id}

        if isinstance(molecular_tests, list):
            gene_results = {}  # Store results for each gene
            for test in molecular_tests:
                if isinstance(test, dict) and 'gene_symbol' in test:
                    gene_symbol = test['gene_symbol']
                    if 'test_result' in test:
                        test_result = test['test_result'].lower()
                        if test_result in ['positive', 'negative']:
                            gene_results.setdefault(gene_symbol, []).append(test_result)

            for gene_symbol, results in gene_results.items():
                if 'positive' in results:
                    # gene_data[gene_symbol] = 1  # Represent positive as 1
                    gene_data[gene_symbol] = "positive"
                elif 'negative' in results:
                    # gene_data[gene_symbol] = 0  # Represent negative as 0
                    gene_data[gene_symbol] = "negative"

                # If no positive or negative results are found, leave it as NaN.
                # You can also assign a default value here if needed.
                # else:
                #    gene_data[gene_symbol] = np.nan


        extracted_data.append(gene_data)

    return pd.DataFrame(extracted_data).fillna(np.nan)


In [ ]:
genesymbol = extract_gene_data(clinical)

In [ ]:
genesymbol

,submitter_id,ESR1,ERBB2,PGR
0,TCGA-E2-A1IU,positive,negative,positive
1,TCGA-A1-A0SB,positive,negative,negative
2,TCGA-A2-A04W,negative,positive,negative
3,TCGA-AN-A0AM,positive,negative,negative
4,TCGA-LL-A440,positive,negative,positive
...,...,...,...,...
1093,TCGA-A2-A0CP,positive,negative,positive
1094,TCGA-PL-A8LX,NaN,NaN,NaN
1095,TCGA-A2-A3XZ,negative,positive,negative
1096,TCGA-E9-A295,positive,positive,positive


In [ ]:
genesymbol.isnull().sum()

,0
submitter_id,0
ESR1,52
ERBB2,138
PGR,55


In [ ]:
1098-138

960

# Step 2: Split the submitter id from the manifest

In [ ]:
manifest['submitter_id'] = manifest['filename'].str[:12]

In [ ]:
manifest["slide_id"] = manifest["filename"].str.replace(r"\.svs$", "", regex=True)

In [ ]:
manifest.head()

,id,filename,md5,size,state,submitter_id,slide_id
0,aa3f0051-ea7f-47f2-a7aa-315c1bdc3f18,TCGA-BH-A0C1-01Z-00-DX1.21FE357E-B182-4397-BFE...,8a43d85058bb6bc2fc00d485cf92e1c1,992971003,released,TCGA-BH-A0C1,TCGA-BH-A0C1-01Z-00-DX1.21FE357E-B182-4397-BFE...
1,a779fff9-b54a-4c18-936f-43f75119b9ad,TCGA-EW-A1P1-01Z-00-DX1.4B670029-4B3B-4D76-8EA...,f90e7b66d9c64ebe9601ea5dbf48a709,1406597123,released,TCGA-EW-A1P1,TCGA-EW-A1P1-01Z-00-DX1.4B670029-4B3B-4D76-8EA...
2,e5806a73-54c4-43d1-a44b-db5f43c8e832,TCGA-E9-A5FL-01Z-00-DX1.FB810D6A-303E-45DF-BEF...,974165ecf6416fbee667776e21e4e489,841538215,released,TCGA-E9-A5FL,TCGA-E9-A5FL-01Z-00-DX1.FB810D6A-303E-45DF-BEF...
3,2449ff02-6925-4f25-9074-7c5fbeab0bd2,TCGA-AC-A23G-01Z-00-DX1.2F0326F7-6B77-4B3F-B4F...,69577c95f8b87e992df71f0bcf3d0635,41783236,released,TCGA-AC-A23G,TCGA-AC-A23G-01Z-00-DX1.2F0326F7-6B77-4B3F-B4F...
4,03a7acd9-178a-4165-9588-fa63e46f3387,TCGA-AO-A1KO-01Z-00-DX1.EEB5E0A0-92B2-42CD-9F7...,6f93a7b1e45bf2d76104a4b04531c235,402997514,released,TCGA-AO-A1KO,TCGA-AO-A1KO-01Z-00-DX1.EEB5E0A0-92B2-42CD-9F7...


In [ ]:
manifest.shape

(1135, 7)

# Step 3: For each genesymbol, create a dataframe and combine with the manifest dataset. And drop all the nan value.

In [ ]:
def combine_dataframes(on_column):
  gene = genesymbol[['submitter_id', on_column]]
  gene_manifest = pd.merge(manifest, gene, on='submitter_id', how='left')
  gene_manifest.dropna(inplace=True)
  return gene_manifest

## ESR1

In [ ]:
# ## ESR1
ESR1 = genesymbol[['submitter_id', 'ESR1']]
# Merge ESR1 data with the manifest dataframe
ESR1_manifest = pd.merge(manifest, ESR1, on='submitter_id', how='left')
ESR1_manifest.dropna(inplace=True)
ESR1_manifest.shape

(1075, 8)

In [ ]:
ESR1_manifest = combine_dataframes('ESR1')
ESR1_manifest.shape

(1075, 8)

## ERBB2

In [ ]:
ERBB2_manifest = combine_dataframes('ERBB2')
ERBB2_manifest.shape

(989, 8)

## PGR

In [ ]:
PGR_manifest = combine_dataframes('PGR')
PGR_manifest.shape

(1072, 8)

# Step 4: Check the Titan BRCA data


## Check the difference slide id and case/submitter id

In [ ]:
tcga_titan = pd.read_csv('./data/tcga_titan.csv')
tcga_titan.rename(columns={'case_id':'submitter_id'}, inplace=True)
tcga_titan.head()

,slide_id,submitter_id,project_id,sex,OncoTreeCode,OncoTreeSiteCode,site,source_site
0,TCGA-H9-A6BY-01Z-00-DX1.304BEB19-3F0A-4676-B07...,TCGA-H9-A6BY,TCGA-PRAD,M,PRAD,PROSTATE,H9,ABS - IUPUI
1,TCGA-VF-A8A8-01Z-00-DX1.4516F2C7-9563-469D-A34...,TCGA-VF-A8A8,TCGA-TGCT,M,NSGCT,TESTIS,VF,University of Pennsylvania
2,TCGA-MT-A67D-01Z-00-DX1.D7A8AEAE-5328-4C19-B29...,TCGA-MT-A67D,TCGA-HNSC,M,HNSC,HEAD_NECK,MT,University of Minnesota
3,TCGA-HT-7603-01Z-00-DX3.78EEF143-9CBB-43E8-B07...,TCGA-HT-7603,TCGA-LGG,M,ODG,BRAIN,HT,Case Western
4,TCGA-E9-A248-01Z-00-DX1.C991932F-39D4-4F2C-91C...,TCGA-E9-A248,TCGA-BRCA,F,IDC,BREAST,E9,Asterand


In [ ]:
brca_titan = tcga_titan[tcga_titan['project_id'] == 'TCGA-BRCA']

In [ ]:
brca_titan

,slide_id,submitter_id,project_id,sex,OncoTreeCode,OncoTreeSiteCode,site,source_site
4,TCGA-E9-A248-01Z-00-DX1.C991932F-39D4-4F2C-91C...,TCGA-E9-A248,TCGA-BRCA,F,IDC,BREAST,E9,Asterand
6,TCGA-A2-A0CQ-01Z-00-DX1.4E5FB4E5-A08C-4C87-A3B...,TCGA-A2-A0CQ,TCGA-BRCA,F,IDC,BREAST,A2,Walter Reed
37,TCGA-GM-A2DI-01Z-00-DX1.5E9715EA-9D2B-49D0-8F1...,TCGA-GM-A2DI,TCGA-BRCA,F,IDC,BREAST,GM,University of Texas MD Anderson Cancer Center
50,TCGA-E2-A152-01Z-00-DX1.B0860DEB-D34B-4C5D-97F...,TCGA-E2-A152,TCGA-BRCA,F,IDC,BREAST,E2,Roswell Park
56,TCGA-BH-A1FH-01Z-00-DX1.F90A691F-B6DB-4C4A-997...,TCGA-BH-A1FH,TCGA-BRCA,F,IDC,BREAST,BH,University of Pittsburgh
...,...,...,...,...,...,...,...,...
11140,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,TCGA-3C-AALK,TCGA-BRCA,F,IDC,BREAST,3C,Columbia University
11162,TCGA-A7-A13E-01Z-00-DX1.891954FF-316A-4562-AA1...,TCGA-A7-A13E,TCGA-BRCA,F,IDC,BREAST,A7,Christiana Healthcare
11178,TCGA-A7-A13E-01Z-00-DX2.1E1262AE-A32D-4814-94A...,TCGA-A7-A13E,TCGA-BRCA,F,IDC,BREAST,A7,Christiana Healthcare
11179,TCGA-HN-A2OB-01Z-00-DX1.14F1FBFB-4540-43CE-9D7...,TCGA-HN-A2OB,TCGA-BRCA,F,ILC,BREAST,HN,Ontario Institute for Cancer Research (OICR)


In [ ]:
brca_titan["slide_id"].iloc[0]

'TCGA-E9-A248-01Z-00-DX1.C991932F-39D4-4F2C-91C3-CE12B5BAEDB1'

### Checking the missing slide id

In [ ]:
brca_titan_slide_ids = set(brca_titan['slide_id'])
manifest_slide_ids = set(manifest['slide_id'])

missing_slide_ids = manifest_slide_ids - brca_titan_slide_ids
print(f"Number of missing slide IDs: {len(missing_slide_ids)}")
print("Missing slide IDs:")

missing_slide_ids


Number of missing slide IDs: 86
Missing slide IDs:


{'TCGA-A1-A0SB-01Z-00-DX1.B34C267B-CAAA-4AB6-AD5C-276C26F997A1',
 'TCGA-A1-A0SE-01Z-00-DX1.04B09232-C6C4-46EF-AA2C-41D078D0A80A',
 'TCGA-A1-A0SK-01Z-00-DX1.A44D70FA-4D96-43F4-9DD7-A61535786297',
 'TCGA-A2-A0YI-01Z-00-DX1.1CF2EC2D-C722-467F-8832-409B823E8D8F',
 'TCGA-A2-A3KD-01Z-00-DX1.E400D529-A0D1-4C78-AD5C-33EB0508B128',
 'TCGA-A2-A4RX-01Z-00-DX1.E2C96C9D-5023-4F41-8CC8-24C6E015B252',
 'TCGA-A2-A4S0-01Z-00-DX1.5878187D-325F-4274-BE41-86BB2BA5F841',
 'TCGA-A2-A4S1-01Z-00-DX1.C69B14F6-5021-4FD8-8F2D-70227A5DA2B1',
 'TCGA-A7-A0CG-01Z-00-DX1.D77019C2-96B1-4EF5-A61E-5F2D5B8D9852',
 'TCGA-A7-A0CG-01Z-00-DX2.81138E90-373A-4C12-B9CA-6EEF5F84A5B5',
 'TCGA-A7-A26F-01Z-00-DX1.7ADE881D-325D-48ED-BA2D-30F589CA8EF2',
 'TCGA-A7-A26G-01Z-00-DX1.29A27673-A16D-4044-BCCF-14BF6236D7D3',
 'TCGA-A7-A3IY-01Z-00-DX1.B4E75265-3B2A-456C-996A-F95B5A645BF8',
 'TCGA-A7-A3J0-01Z-00-DX1.59DB32BC-F59C-43B3-8DDD-4A728EBA8AB8',
 'TCGA-A7-A3RF-01Z-00-DX1.E60647BC-229D-4409-A564-EE0945F912EE',
 'TCGA-A7-A5ZV-01Z-00-DX1

### Checking the missing submitter/case id

In [ ]:
brca_titan_submitter_ids = set(brca_titan['submitter_id'])
manifest_submitter_ids = set(manifest['submitter_id'])

missing_submitter_ids = manifest_submitter_ids - brca_titan_submitter_ids
print(f"Number of missing submitter IDs: {len(missing_submitter_ids)}")
print("Missing submitter IDs:")
missing_submitter_ids


Number of missing submitter IDs: 80
Missing submitter IDs:


{'TCGA-A1-A0SB',
 'TCGA-A1-A0SE',
 'TCGA-A1-A0SK',
 'TCGA-A2-A0YI',
 'TCGA-A2-A3KD',
 'TCGA-A2-A4RX',
 'TCGA-A2-A4S0',
 'TCGA-A2-A4S1',
 'TCGA-A7-A0CG',
 'TCGA-A7-A26F',
 'TCGA-A7-A26G',
 'TCGA-A7-A3IY',
 'TCGA-A7-A3J0',
 'TCGA-A7-A3RF',
 'TCGA-A7-A5ZV',
 'TCGA-A8-A07S',
 'TCGA-A8-A08J',
 'TCGA-A8-A0AD',
 'TCGA-AC-A2QH',
 'TCGA-AC-A2QJ',
 'TCGA-AC-A3YJ',
 'TCGA-AC-A5EI',
 'TCGA-AC-A7VC',
 'TCGA-AC-A8OR',
 'TCGA-AN-A0AJ',
 'TCGA-AO-A03U',
 'TCGA-AO-A125',
 'TCGA-AR-A24M',
 'TCGA-AR-A24O',
 'TCGA-AR-A24X',
 'TCGA-AR-A2LR',
 'TCGA-AR-A5QQ',
 'TCGA-B6-A0RP',
 'TCGA-B6-A0RV',
 'TCGA-B6-A0X0',
 'TCGA-BH-A0BA',
 'TCGA-BH-A0C3',
 'TCGA-BH-A0DL',
 'TCGA-BH-A0WA',
 'TCGA-BH-A1EO',
 'TCGA-BH-A1F2',
 'TCGA-C8-A12X',
 'TCGA-C8-A1HL',
 'TCGA-C8-A8HP',
 'TCGA-C8-A8HQ',
 'TCGA-D8-A1XC',
 'TCGA-D8-A1XS',
 'TCGA-D8-A1XV',
 'TCGA-D8-A27W',
 'TCGA-D8-A73W',
 'TCGA-D9-A148',
 'TCGA-E2-A15G',
 'TCGA-E9-A1N3',
 'TCGA-E9-A1N5',
 'TCGA-E9-A1N9',
 'TCGA-E9-A1NA',
 'TCGA-E9-A1NC',
 'TCGA-E9-A1NE',
 'TCGA-E9-A1NF

### Summary: after checking the image of one missing slide id, I guess the reason that TITAN remove these slides is because there is marker on the slide image.
Example:

1. missing slide id: `TCGA-A1-A0SB-01Z-00-DX1.B34C267B-CAAA-4AB6-AD5C-276C26F997A1`
![](https://drive.google.com/uc?id=1GCsZHTjHs4ypHYpxwEduytjNBCaLhJXd)
2. not missing slide id: `TCGA-E9-A248-01Z-00-DX1.C991932F-39D4-4F2C-91C3-CE12B5BAEDB1`
![](https://drive.google.com/uc?id=1vyk9aYHLvyHFPB3ejyg1h4xw66DRH4Ll)




## Test if we use TITAN BRCA data, can we get the same patients and WSIs number that showed in the TITAN article
| Dataset          | Tissue Site | Target | Label-level | # Classes | # Patients | # WSIs | Metric      |
|------------------|-------------|--------|-------------|-----------|------------|--------|-------------|
| TCGA-BRCA        | Breast      | ER     | Patient     | 2         | 937        | 996    | AUROC       |      |
| TCGA-BRCA        | Breast      | PR     | Patient     | 2         | 934        | 993    | AUROC       |      |
| TCGA-BRCA        | Breast      | HER2   | Patient     | 2         | 647        | 693    | AUROC       |


In [ ]:
def test_combine_dataframes(on_column):
  gene = genesymbol[['submitter_id', on_column]]
  gene_titan = pd.merge(brca_titan, gene, on='submitter_id', how='left')
  gene_titan.dropna(inplace=True)
  return gene_titan

### ESR1 (ER)

In [ ]:
ESR1_manifest = test_combine_dataframes('ESR1')
ESR1_manifest.shape

(995, 9)

In [ ]:
ESR1_manifest.to_csv('./data/ESR1_manifest.csv', index=False)

### ERBB2 (HER2)

In [ ]:
ERBB2_manifest = test_combine_dataframes('ERBB2')
ERBB2_manifest.shape

(914, 9)

### PGR (PR)

In [ ]:
PGR_manifest = test_combine_dataframes('PGR')
PGR_manifest.head(10)

,slide_id,submitter_id,project_id,sex,OncoTreeCode,OncoTreeSiteCode,site,source_site,PGR
1,TCGA-A2-A0CQ-01Z-00-DX1.4E5FB4E5-A08C-4C87-A3B...,TCGA-A2-A0CQ,TCGA-BRCA,F,IDC,BREAST,A2,Walter Reed,positive
2,TCGA-GM-A2DI-01Z-00-DX1.5E9715EA-9D2B-49D0-8F1...,TCGA-GM-A2DI,TCGA-BRCA,F,IDC,BREAST,GM,University of Texas MD Anderson Cancer Center,negative
3,TCGA-E2-A152-01Z-00-DX1.B0860DEB-D34B-4C5D-97F...,TCGA-E2-A152,TCGA-BRCA,F,IDC,BREAST,E2,Roswell Park,negative
4,TCGA-BH-A1FH-01Z-00-DX1.F90A691F-B6DB-4C4A-997...,TCGA-BH-A1FH,TCGA-BRCA,F,IDC,BREAST,BH,University of Pittsburgh,negative
5,TCGA-BH-A0DS-01Z-00-DX1.38E82A1F-21B9-4B97-A30...,TCGA-BH-A0DS,TCGA-BRCA,F,IDC,BREAST,BH,University of Pittsburgh,positive
6,TCGA-C8-A130-01Z-00-DX1.F6810204-C35A-4461-A18...,TCGA-C8-A130,TCGA-BRCA,F,IDC,BREAST,C8,ILSbio,positive
7,TCGA-BH-A0BP-01Z-00-DX1.63A87C1D-87FA-494D-983...,TCGA-BH-A0BP,TCGA-BRCA,F,IDC,BREAST,BH,University of Pittsburgh,positive
8,TCGA-A2-A1FW-01Z-00-DX1.DC2BC5D7-E57B-444E-833...,TCGA-A2-A1FW,TCGA-BRCA,F,IDC,BREAST,A2,Walter Reed,negative
10,TCGA-E2-A150-01Z-00-DX1.55643D23-83E6-4058-8EA...,TCGA-E2-A150,TCGA-BRCA,F,IDC,BREAST,E2,Roswell Park,negative
11,TCGA-A8-A0A4-01Z-00-DX1.06C38F98-F34B-49F0-AFB...,TCGA-A8-A0A4,TCGA-BRCA,F,ILC,BREAST,A8,Indivumed,positive


In [ ]:
len(PGR_manifest["submitter_id"].unique())

934

In [ ]:

PGR_manifest.to_csv('./data/PGR_manifest.csv', index=False)

so from the result, I can accept the difference between the TITAN and my data on ESR1 and PGR is because the missing slides.

But for the EBRR2, I need to check the ERBB2 data again

# Step 5: Recheck the ERBB2 data
5/15/2025

In [ ]:
clinical.head()

,disease_type,project,submitter_id,days_to_consent,diagnoses,consent_type,demographic,primary_site,updated_datetime,case_id,follow_ups,index_date,state,lost_to_followup,exposures,molecular_tests
0,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},TCGA-E2-A1IU,-34.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': 'd14426b2-e0a0-519a-bea6-4f...,Breast,2025-01-06T00:20:17.681998-06:00,001cef41-ff86-4d3f-a140-a647ac4b10a1,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[{'molecular_analysis_method': 'IHC', 'test_re..."
1,Adenomas and Adenocarcinomas,{'project_id': 'TCGA-BRCA'},TCGA-A1-A0SB,76.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': 'fa693617-eab9-502c-9a16-0e...,Breast,2025-01-05T21:27:11.301925-06:00,0045349c-69d9-4306-a403-c9c1fa836644,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[{'molecular_analysis_method': 'IHC', 'test_re..."
2,Adnexal and Skin Appendage Neoplasms,{'project_id': 'TCGA-BRCA'},TCGA-A2-A04W,19.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': '6562e53d-99fd-54ed-b8e1-51...,Breast,2025-01-06T07:31:06.617417-06:00,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,"[{'timepoint_category': 'Follow-up', 'follow_u...",Diagnosis,released,No,NaN,"[{'molecular_analysis_method': 'FISH', 'specia..."
3,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},TCGA-AN-A0AM,0.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': '7366952a-e8e7-56ec-9867-23...,Breast,2025-01-05T22:52:03.042081-06:00,00a2d166-78c9-4687-a195-3d6315c27574,"[{'timepoint_category': 'Follow-up', 'follow_u...",Diagnosis,released,Yes,NaN,"[{'molecular_analysis_method': 'IHC', 'test_re..."
4,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},TCGA-LL-A440,11.0,"[{'ajcc_pathologic_t': 'Tis (DCIS)', 'morpholo...",Informed Consent,{'demographic_id': 'f5229922-62e2-51d5-ba4e-94...,Breast,2025-01-05T22:47:37.828549-06:00,00b11ca8-8540-4a3d-b602-ec754b00230b,"[{'timepoint_category': 'Follow-up', 'follow_u...",Diagnosis,released,No,NaN,"[{'molecular_analysis_method': 'FISH', 'specia..."


In [ ]:
def extract_erbb2_results(clinical_data):
    """
    Extracts all ERBB2 test results from the follow_ups of clinical data.
    A submitter may have multiple ERBB2 results, so the results are returned as a list.
    """
    erbb2_results = []
    for index, row in clinical_data.iterrows():
        submitter_id = row['submitter_id']
        molecular_tests = row['molecular_tests']
        erbb2_values = []

        if isinstance(molecular_tests, list):
            for test in molecular_tests:
                if isinstance(test, dict) and 'gene_symbol' in test and test['gene_symbol'] == 'ERBB2':
                    if 'test_result' in test:
                        erbb2_values.append(test['test_result'])

        erbb2_results.append({'submitter_id': submitter_id, 'ERBB2_results': erbb2_values})

    return pd.DataFrame(erbb2_results)

erbb2_df = extract_erbb2_results(clinical)
erbb2_df


,submitter_id,ERBB2_results
0,TCGA-E2-A1IU,[Negative]
1,TCGA-A1-A0SB,[Negative]
2,TCGA-A2-A04W,"[Test Value Reported, Positive, Positive]"
3,TCGA-AN-A0AM,[Negative]
4,TCGA-LL-A440,"[Test Value Reported, Negative, Copy Number Re..."
...,...,...
1093,TCGA-A2-A0CP,[Negative]
1094,TCGA-PL-A8LX,[]
1095,TCGA-A2-A3XZ,[Positive]
1096,TCGA-E9-A295,[Positive]


### Delete the nan value

In [ ]:
#For the ERBB2_results column of erbb2_df, delete the rows with a length of 0.
erbb2_df = erbb2_df[erbb2_df['ERBB2_results'].str.len() != 0]

In [ ]:
erbb2_df

,submitter_id,ERBB2_results
0,TCGA-E2-A1IU,[Negative]
1,TCGA-A1-A0SB,[Negative]
2,TCGA-A2-A04W,"[Test Value Reported, Positive, Positive]"
3,TCGA-AN-A0AM,[Negative]
4,TCGA-LL-A440,"[Test Value Reported, Negative, Copy Number Re..."
...,...,...
1092,TCGA-B6-A40C,[Negative]
1093,TCGA-A2-A0CP,[Negative]
1095,TCGA-A2-A3XZ,[Positive]
1096,TCGA-E9-A295,[Positive]


In [ ]:
# For the ERBB2_results column of erbb2_df, delete the rows that only contain 'Equvical'
erbb2_df = erbb2_df[~erbb2_df['ERBB2_results'].apply(lambda x: all(item == 'Equivocal' for item in x if isinstance(x, list) and len(x) > 0))]

In [ ]:
erbb2_df

,submitter_id,ERBB2_results
0,TCGA-E2-A1IU,[Negative]
1,TCGA-A1-A0SB,[Negative]
2,TCGA-A2-A04W,"[Test Value Reported, Positive, Positive]"
3,TCGA-AN-A0AM,[Negative]
4,TCGA-LL-A440,"[Test Value Reported, Negative, Copy Number Re..."
...,...,...
1091,TCGA-BH-A0HP,[Negative]
1092,TCGA-B6-A40C,[Negative]
1093,TCGA-A2-A0CP,[Negative]
1095,TCGA-A2-A3XZ,[Positive]


In [ ]:
erbb2_df["ERBB2_results"].iloc[2]

['Test Value Reported', 'Positive', 'Positive']

In [ ]:
# drop the row where erbb2_df["ERBB2_results"] contains "Equivocal"

# Drop rows where 'ERBB2_results' contains "Equivocal"
erbb2_df = erbb2_df[~erbb2_df['ERBB2_results'].apply(lambda x: "Equivocal" in x if isinstance(x, list) else False)]
erbb2_df


,submitter_id,ERBB2_results
0,TCGA-E2-A1IU,[Negative]
1,TCGA-A1-A0SB,[Negative]
2,TCGA-A2-A04W,"[Test Value Reported, Positive, Positive]"
3,TCGA-AN-A0AM,[Negative]
5,TCGA-A7-A26E,[Negative]
...,...,...
1091,TCGA-BH-A0HP,[Negative]
1092,TCGA-B6-A40C,[Negative]
1093,TCGA-A2-A0CP,[Negative]
1095,TCGA-A2-A3XZ,[Positive]


In [ ]:
# drop drop the row where erbb2_df["ERBB2_results"] not ending with "Positive " or "Negative"

# Drop rows where 'ERBB2_results' does not end with "Positive" or "Negative"
erbb2_df = erbb2_df[erbb2_df['ERBB2_results'].apply(lambda x: isinstance(x, list) and (x[-1].endswith("Positive") or x[-1].endswith("Negative")) if x else False)]
erbb2_df


,submitter_id,ERBB2_results
0,TCGA-E2-A1IU,[Negative]
1,TCGA-A1-A0SB,[Negative]
2,TCGA-A2-A04W,"[Test Value Reported, Positive, Positive]"
3,TCGA-AN-A0AM,[Negative]
5,TCGA-A7-A26E,[Negative]
...,...,...
1091,TCGA-BH-A0HP,[Negative]
1092,TCGA-B6-A40C,[Negative]
1093,TCGA-A2-A0CP,[Negative]
1095,TCGA-A2-A3XZ,[Positive]


In [ ]:
# drop drop the row where erbb2_df["ERBB2_results"] contain both with "Positive " and "Negative"

# Drop rows where 'ERBB2_results' contains both "Positive" and "Negative"
erbb2_df = erbb2_df[~erbb2_df['ERBB2_results'].apply(lambda x: "Positive" in [item.strip() for item in x] and "Negative" in [item.strip() for item in x] if isinstance(x, list) else False)]
erbb2_df


,submitter_id,ERBB2_results
0,TCGA-E2-A1IU,[Negative]
1,TCGA-A1-A0SB,[Negative]
2,TCGA-A2-A04W,"[Test Value Reported, Positive, Positive]"
3,TCGA-AN-A0AM,[Negative]
5,TCGA-A7-A26E,[Negative]
...,...,...
1091,TCGA-BH-A0HP,[Negative]
1092,TCGA-B6-A40C,[Negative]
1093,TCGA-A2-A0CP,[Negative]
1095,TCGA-A2-A3XZ,[Positive]


In [ ]:
# extract the last item of ERBB2_result column out and drop it, keep the ERBB2 column
erbb2_df['ERBB2'] = erbb2_df['ERBB2_results'].apply(lambda x: x[-1] if isinstance(x, list) and x else None)
erbb2_df.drop(columns=['ERBB2_results'], inplace=True)
erbb2_df['ERBB2'] = erbb2_df['ERBB2'].str.lower()
erbb2_df


<ipython-input-133-c23e4ed4b320>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erbb2_df['ERBB2'] = erbb2_df['ERBB2_results'].apply(lambda x: x[-1] if isinstance(x, list) and x else None)
<ipython-input-133-c23e4ed4b320>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erbb2_df.drop(columns=['ERBB2_results'], inplace=True)
<ipython-input-133-c23e4ed4b320>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,submitter_id,ERBB2
0,TCGA-E2-A1IU,negative
1,TCGA-A1-A0SB,negative
2,TCGA-A2-A04W,positive
3,TCGA-AN-A0AM,negative
5,TCGA-A7-A26E,negative
...,...,...
1091,TCGA-BH-A0HP,negative
1092,TCGA-B6-A40C,negative
1093,TCGA-A2-A0CP,negative
1095,TCGA-A2-A3XZ,positive


In [ ]:
def test_combine_dataframes(on_column):
  gene = erbb2_df[['submitter_id', on_column]]
  gene_titan = pd.merge(brca_titan, gene, on='submitter_id', how='left')
  gene_titan.dropna(inplace=True)
  return gene_titan

In [ ]:
ERBB2_manifest = test_combine_dataframes('ERBB2')
ERBB2_manifest.shape

(691, 9)

In [ ]:
ERBB2_manifest.to_csv('./data/ERBB2_manifest.csv', index=False)

In [ ]:
ERBB2_manifest

,slide_id,submitter_id,project_id,sex,OncoTreeCode,OncoTreeSiteCode,site,source_site,ERBB2
2,TCGA-GM-A2DI-01Z-00-DX1.5E9715EA-9D2B-49D0-8F1...,TCGA-GM-A2DI,TCGA-BRCA,F,IDC,BREAST,GM,University of Texas MD Anderson Cancer Center,negative
3,TCGA-E2-A152-01Z-00-DX1.B0860DEB-D34B-4C5D-97F...,TCGA-E2-A152,TCGA-BRCA,F,IDC,BREAST,E2,Roswell Park,positive
5,TCGA-BH-A0DS-01Z-00-DX1.38E82A1F-21B9-4B97-A30...,TCGA-BH-A0DS,TCGA-BRCA,F,IDC,BREAST,BH,University of Pittsburgh,negative
7,TCGA-BH-A0BP-01Z-00-DX1.63A87C1D-87FA-494D-983...,TCGA-BH-A0BP,TCGA-BRCA,F,IDC,BREAST,BH,University of Pittsburgh,negative
8,TCGA-A2-A1FW-01Z-00-DX1.DC2BC5D7-E57B-444E-833...,TCGA-A2-A1FW,TCGA-BRCA,F,IDC,BREAST,A2,Walter Reed,negative
...,...,...,...,...,...,...,...,...,...
1037,TCGA-A7-A0DB-01Z-00-DX1.9CE855BC-0C37-43FB-880...,TCGA-A7-A0DB,TCGA-BRCA,F,IDC,BREAST,A7,Christiana Healthcare,negative
1039,TCGA-A7-A6VX-01Z-00-DX2.9EE94B59-6A2C-4507-AA4...,TCGA-A7-A6VX,TCGA-BRCA,F,IDC,BREAST,A7,Christiana Healthcare,negative
1042,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,TCGA-3C-AALI,TCGA-BRCA,F,IDC,BREAST,3C,Columbia University,positive
1044,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,TCGA-3C-AALK,TCGA-BRCA,F,IDC,BREAST,3C,Columbia University,positive


# Step 6: Testing the FISH in the ERBB2 and redesign the ERBB2 cleaning algorithm

5/16/2025

In [ ]:
ERBB2_dataframe = []
for tests_list in clinical['molecular_tests']:
  for test in tests_list:
    if test.get('gene_symbol') == 'ERBB2':
      ERBB2_dataframe.append(test)

ERBB2_dataframe = pd.DataFrame(ERBB2_dataframe)
ERBB2_dataframe.head(3)


,molecular_analysis_method,test_result,molecular_test_id,updated_datetime,gene_symbol,submitter_id,state,created_datetime,specialized_molecular_test,chromosome,test_value,variant_type,cell_count,copy_number,staining_intensity_value,test_value_range
0,IHC,Negative,cb8fecfa-0f96-40cb-b871-6b030ace4c5e,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-E2-A1IU_molecular_test3,released,2025-01-06T00:20:17.681998-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IHC,Negative,208069ac-9a46-46fa-9445-3f03c2d4da2f,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-A1-A0SB_molecular_test3,released,2025-01-05T21:27:11.301925-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FISH,Test Value Reported,1fb44705-e07b-4757-9d2d-b2b07f4549b8,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-A2-A04W_molecular_test5,released,2025-01-06T07:31:06.617417-06:00,Signal ratio,chr17,3.2,NaN,NaN,NaN,NaN,NaN


In [ ]:
def extract_submitter_id(submitter_id):
  """Extracts the base submitter ID from a string."""
  return submitter_id.split('_')[0]

# Apply the function to the submitter_id column if it exists
if 'submitter_id' in ERBB2_dataframe.columns:
  ERBB2_dataframe['base_submitter_id'] = ERBB2_dataframe['submitter_id'].apply(extract_submitter_id)
ERBB2_dataframe.head(3)


,molecular_analysis_method,test_result,molecular_test_id,updated_datetime,gene_symbol,submitter_id,state,created_datetime,specialized_molecular_test,chromosome,test_value,variant_type,cell_count,copy_number,staining_intensity_value,test_value_range,base_submitter_id
0,IHC,Negative,cb8fecfa-0f96-40cb-b871-6b030ace4c5e,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-E2-A1IU_molecular_test3,released,2025-01-06T00:20:17.681998-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-E2-A1IU
1,IHC,Negative,208069ac-9a46-46fa-9445-3f03c2d4da2f,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-A1-A0SB_molecular_test3,released,2025-01-05T21:27:11.301925-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-A1-A0SB
2,FISH,Test Value Reported,1fb44705-e07b-4757-9d2d-b2b07f4549b8,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-A2-A04W_molecular_test5,released,2025-01-06T07:31:06.617417-06:00,Signal ratio,chr17,3.2,NaN,NaN,NaN,NaN,NaN,TCGA-A2-A04W


In [ ]:
ERBB2_dataframe['test'] = ERBB2_dataframe['submitter_id'].apply(lambda x: int(re.search(r'\d+$', x).group()) if re.search(r'\d+$', x) else None)
ERBB2_dataframe.head(3)

,molecular_analysis_method,test_result,molecular_test_id,updated_datetime,gene_symbol,submitter_id,state,created_datetime,specialized_molecular_test,chromosome,test_value,variant_type,cell_count,copy_number,staining_intensity_value,test_value_range,base_submitter_id,test
0,IHC,Negative,cb8fecfa-0f96-40cb-b871-6b030ace4c5e,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-E2-A1IU_molecular_test3,released,2025-01-06T00:20:17.681998-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-E2-A1IU,3.0
1,IHC,Negative,208069ac-9a46-46fa-9445-3f03c2d4da2f,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-A1-A0SB_molecular_test3,released,2025-01-05T21:27:11.301925-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-A1-A0SB,3.0
2,FISH,Test Value Reported,1fb44705-e07b-4757-9d2d-b2b07f4549b8,2025-01-08T12:58:05.350809-06:00,ERBB2,TCGA-A2-A04W_molecular_test5,released,2025-01-06T07:31:06.617417-06:00,Signal ratio,chr17,3.2,NaN,NaN,NaN,NaN,NaN,TCGA-A2-A04W,5.0


In [ ]:
ERBB2_dataframe.drop(columns=['molecular_test_id', 'updated_datetime', 'state', 'created_datetime', 'specialized_molecular_test', 'chromosome', 'variant_type', 'cell_count', 'copy_number', 'test_value_range'], inplace=True)

In [ ]:
ERBB2_dataframe[ERBB2_dataframe['base_submitter_id'] == 'TCGA-AO-A126'].sort_values(by='test')

,molecular_analysis_method,test_result,gene_symbol,submitter_id,test_value,staining_intensity_value,base_submitter_id,test
1389,IHC,Negative,ERBB2,TCGA-AO-A126_molecular_test3,NaN,1+,TCGA-AO-A126,3.0
1390,FISH,Negative,ERBB2,TCGA-AO-A126_molecular_test4,NaN,NaN,TCGA-AO-A126,4.0
1388,FISH,Copy Number Reported,ERBB2,TCGA-AO-A126_molecular_test5,NaN,NaN,TCGA-AO-A126,5.0
1387,FISH,Test Value Reported,ERBB2,TCGA-AO-A126_molecular_test7,1.1,NaN,TCGA-AO-A126,7.0
1392,IHC,Negative,ERBB2,TCGA-AO-A126_molecular_test10,NaN,0,TCGA-AO-A126,10.0
1391,FISH,Negative,ERBB2,TCGA-AO-A126_molecular_test11,NaN,NaN,TCGA-AO-A126,11.0
1393,FISH,Test Value Reported,ERBB2,TCGA-AO-A126_molecular_test12,1.3,NaN,TCGA-AO-A126,12.0


In [ ]:
fish_positive_negative = ERBB2_dataframe[ERBB2_dataframe['test_result'].isin(['Positive', 'Negative'])]
TCGA_AO_A126 = fish_positive_negative[fish_positive_negative['base_submitter_id'] == 'TCGA-AO-A126'].sort_values(by='test')

In [ ]:
TCGA_AO_A126["test_value"].max()

nan

In [ ]:
def process_erbb2_results(ERBB2_dataframe):
    """
    Processes ERBB2 molecular test results for each base_submitter_id.

    For each base_submitter_id, the function sorts the tests by 'test' and
    determines the final test_result based on the following logic:
    - If only IHC method is present, the result is the test_result for that row.
    - If FISH methods are present, the result is the last 'Positive' or 'Negative'
      result among the FISH tests.
    """
    final_results = {}
    for submitter_id, group in ERBB2_dataframe.groupby('base_submitter_id'):
        sorted_group = group.sort_values(by='test')

        ihc_results = sorted_group[sorted_group['molecular_analysis_method'] == 'IHC']
        fish_results = sorted_group[sorted_group['molecular_analysis_method'] == 'FISH']

        if len(fish_results) == 0 and len(ihc_results) > 0:
            # Only IHC present, take the result from the first (and likely only) IHC row after sorting
            final_results[submitter_id] = ihc_results.iloc[0]['test_result']
        elif len(fish_results) > 0:
            # FISH results present, find the last Positive or Negative result
            fish_positive_negative = fish_results[fish_results['test_result'].isin(['Positive', 'Negative'])]
            if len(fish_positive_negative) > 0:
              if len(fish_positive_negative["test_result"].value_counts()) > 1:
                print("Here is a conflict: ",fish_positive_negative["test_result"].value_counts())
                # if tere is both positive and negative set the conflict or just use the last one
                # print(fish_positive_negative["test_result"].value_counts())
                final_results[submitter_id] = "Conflict"
              else:
                final_results[submitter_id] = fish_positive_negative.iloc[-1]['test_result']
            else:
              # if there is no negative or positive
              # print(fish_results["test_value"].value_counts())
              if fish_results["test_value"].sum() > 0:
                # print("Here is the test_result",fish_results["test_result"])
                # print()
                # print("there is test value in the fish result: ", fish_results["test_value"])
                # ERBB2 ratio value >2 positive, <2 negative, here we used the max value in here
                # if the total test value is larger than 0, than check the max value
                if fish_results["test_value"].max() >= 2:
                  final_results[submitter_id] = "Positive"
                elif fish_results["test_value"].max() < 2:
                  final_results[submitter_id] = "Negative"
                else:
                  final_results[submitter_id] = None
              else:
                final_results[submitter_id] = None
        else:
            final_results[submitter_id] = None # Or some other indicator

    final_erbb2_df = pd.DataFrame.from_dict(final_results, orient='index', columns=['final_ERBB2_result'])
    final_erbb2_df.index.name = 'base_submitter_id'
    final_erbb2_df.reset_index(inplace=True)
    return final_erbb2_df



# Example usage:
final_erbb2_df = process_erbb2_results(ERBB2_dataframe)
final_erbb2_df


Here is a conflict:  test_result
Positive    1
Negative    1
Name: count, dtype: int64


,base_submitter_id,final_ERBB2_result
0,TCGA-3C-AAAU,Negative
1,TCGA-3C-AALI,Positive
2,TCGA-3C-AALJ,Equivocal
3,TCGA-3C-AALK,Positive
4,TCGA-4H-AAAK,Equivocal
...,...,...
979,TCGA-WT-AB44,Negative
980,TCGA-XX-A899,Negative
981,TCGA-XX-A89A,Negative
982,TCGA-Z7-A8R5,Negative


In [ ]:
final_erbb2_df["final_ERBB2_result"].value_counts()

,count
final_ERBB2_result,
Negative,772
Positive,182
Equivocal,22
Conflict,1


In [ ]:
final_erbb2_df = final_erbb2_df[final_erbb2_df["final_ERBB2_result"] != "Equivocal"]
final_erbb2_df = final_erbb2_df[final_erbb2_df["final_ERBB2_result"] != "Conflict"]
final_erbb2_df["final_ERBB2_result"].value_counts()

,count
final_ERBB2_result,
Negative,772
Positive,182


In [ ]:
final_erbb2_df.rename(columns={'base_submitter_id': 'submitter_id', 'final_ERBB2_result': 'ERBB2'}, inplace=True)

## Get the ERBB2 filter data

In [ ]:
ERBB2_manifest = pd.merge(brca_titan, final_erbb2_df, on='submitter_id', how='left')
ERBB2_manifest.dropna(inplace=True)
ERBB2_manifest['ERBB2'] = ERBB2_manifest['ERBB2'].str.lower()
ERBB2_manifest.shape

(909, 9)

In [ ]:
ERBB2_manifest.to_csv('./data/ERBB2_manifest_v2.csv', index=False)

## Check on the IHC

In [ ]:
IHC = ERBB2_dataframe[ERBB2_dataframe["molecular_analysis_method"] == "IHC"]
IHC.head()

,molecular_analysis_method,test_result,gene_symbol,submitter_id,test_value,staining_intensity_value,base_submitter_id,test
0,IHC,Negative,ERBB2,TCGA-E2-A1IU_molecular_test3,NaN,NaN,TCGA-E2-A1IU,3.0
1,IHC,Negative,ERBB2,TCGA-A1-A0SB_molecular_test3,NaN,NaN,TCGA-A1-A0SB,3.0
3,IHC,Positive,ERBB2,TCGA-A2-A04W_molecular_test3,NaN,NaN,TCGA-A2-A04W,3.0
5,IHC,Negative,ERBB2,TCGA-AN-A0AM_molecular_test3,NaN,NaN,TCGA-AN-A0AM,3.0
9,IHC,Equivocal,ERBB2,TCGA-LL-A440_molecular_test3,NaN,2+,TCGA-LL-A440,3.0


In [ ]:
IHC[IHC['base_submitter_id'] == 'TCGA-4H-AAAK']

,molecular_analysis_method,test_result,gene_symbol,submitter_id,test_value,staining_intensity_value,base_submitter_id,test
696,IHC,Equivocal,ERBB2,TCGA-4H-AAAK_molecular_test3,NaN,2+,TCGA-4H-AAAK,3.0


## Check on the FISH

In [ ]:
FISH = ERBB2_dataframe[ERBB2_dataframe["molecular_analysis_method"] == "FISH"]
FISH.head(3)

,molecular_analysis_method,test_result,gene_symbol,submitter_id,test_value,staining_intensity_value,base_submitter_id,test
2,FISH,Test Value Reported,ERBB2,TCGA-A2-A04W_molecular_test5,3.2,NaN,TCGA-A2-A04W,5.0
4,FISH,Positive,ERBB2,TCGA-A2-A04W_molecular_test4,NaN,NaN,TCGA-A2-A04W,4.0
6,FISH,Test Value Reported,ERBB2,TCGA-LL-A440_molecular_test7,1.7,NaN,TCGA-LL-A440,7.0


In [ ]:
FISH_count = pd.DataFrame(FISH["base_submitter_id"].value_counts())
FISH_count.rename(columns={"base_submitter_id": "FISH_count"}, inplace=True)
FISH_count


,count
base_submitter_id,
TCGA-AO-A126,5
TCGA-E2-A1IE,4
TCGA-E2-A1LK,4
TCGA-AO-A03P,4
TCGA-A2-A04P,4
...,...
TCGA-AR-A1AM,1
TCGA-E2-A576,1
TCGA-A8-A091,1


In [ ]:
FISH_count[FISH_count["count"] >= 2]

,count
base_submitter_id,
TCGA-AO-A126,5
TCGA-E2-A1IE,4
TCGA-E2-A1LK,4
TCGA-AO-A03P,4
TCGA-A2-A04P,4
...,...
TCGA-AQ-A04H,2
TCGA-GM-A5PX,2
TCGA-A2-A04R,2
